@Author:Vijay Kumar M N<br>
@Date: 2024-10-10<br>
@Last Modified by:Vijay Kumar M N<br>
@Last Modified: 2024-10-10<br>
@Title :Python Program on covid dataset operation by pyspark</br>

In [105]:
from pyspark.sql import  SparkSession
from pyspark.sql.functions import expr,col
spark=(SparkSession.builder.appName("Covid_operations").master("local[*]").getOrCreate())

In [106]:
spark

In [107]:
#reading a files
worldometer = spark.read.csv("C://Users//Admin//Desktop//sql//worldometer_data.csv", header=True, inferSchema=True)
covid_19_india= spark.read.csv("C://Users//Admin//Desktop//sql//Covid Problem//covid_19_india.csv", header=True, inferSchema=True)
statewisetesting= spark.read.csv("C://Users//Admin//Desktop\sql//Covid Problem//StatewiseTestingDetails.csv", header=True, inferSchema=True)
covid_19_clean_complete = spark.read.csv("C://Users//Admin//Desktop//sql//covid_19_clean_complete.csv", header=True, inferSchema=True)


<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_13932\1667553111.py:3: SyntaxWarning: invalid escape sequence '\s'
  statewisetesting= spark.read.csv("C://Users//Admin//Desktop\sql//Covid Problem//StatewiseTestingDetails.csv", header=True, inferSchema=True)


In [108]:
#creating a temp views
worldometer.createOrReplaceTempView("worldometer")
covid_19_india.createOrReplaceTempView("covid_19_india")
statewisetesting.createOrReplaceTempView("statewisetestings")
covid_19_clean_complete.createOrReplaceTempView("covid_19_clean_complete")

1. To find out the death percentage locally and globally

In [109]:
#to find the death percentage globally
worldometer.printSchema()
worldometer.show()


root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDe

In [110]:
#to find the death percentage globally
covid_globally_death_percentage = """
SELECT 
    sum(TotalDeaths) AS TotalDeaths,
    sum(TotalCases) AS TotalCases,
    ROUND(sum(TotalDeaths) / sum(TotalCases) * 100, 2) AS DeathPercentage
FROM 
    Global_data
"""
spark.sql(covid_globally_death_percentage).show()


+-----------+----------+---------------+
|TotalDeaths|TotalCases|DeathPercentage|
+-----------+----------+---------------+
|     713007|  19169166|           3.72|
+-----------+----------+---------------+



In [111]:
#reading a file and showing schema
covid_19_india.printSchema()
covid_19_india.show()

root
 |-- Sno: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- State/UnionTerritory: string (nullable = true)
 |-- ConfirmedIndianNational: string (nullable = true)
 |-- ConfirmedForeignNational: string (nullable = true)
 |-- Cured: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Confirmed: integer (nullable = true)

+---+----------+-------+--------------------+-----------------------+------------------------+-----+------+---------+
|Sno|      Date|   Time|State/UnionTerritory|ConfirmedIndianNational|ConfirmedForeignNational|Cured|Deaths|Confirmed|
+---+----------+-------+--------------------+-----------------------+------------------------+-----+------+---------+
|  1|2020-01-30|6:00 PM|              Kerala|                      1|                       0|    0|     0|        1|
|  2|2020-01-31|6:00 PM|              Kerala|                      1|                       0|    0|     0|        1|
|  3|2020-02-01|

In [112]:
#to find out the death percentage locally
covid_locally_death_percentage = """
SELECT 
    
	SUM(Confirmed) AS Total_Confirmed,
    SUM(Deaths) AS Total_Deaths,
    ROUND((SUM(Deaths) / SUM(Confirmed) * 100), 2) as percentage_locally
FROM 
    covid_19_india

"""
spark.sql(covid_locally_death_percentage).show()


+---------------+------------+------------------+
|Total_Confirmed|Total_Deaths|percentage_locally|
+---------------+------------+------------------+
|     5451678687|    73389005|              1.35|
+---------------+------------+------------------+



2. To find out the infected population percentage locally and globally

In [113]:
#globally infected percentage
worldometer.show()
worldometer.printSchema()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|    NULL|     162804|     NULL|       2576668|        NULL|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    NULL|      98644|     NULL|       2047660|        NULL|     771258|         

In [114]:
#to find out the infected percentage globally
covid_infected_death_percentage = """
select 
	sum(TotalCases) as Total_Infected,
	sum(Population) as Total_population, 
	ROUND((SUM (TotalCases) / SUM(Population) * 100), 2) AS InfectionPercentage
from 
	worldometer;

"""
spark.sql(covid_infected_death_percentage).show()


+--------------+----------------+-------------------+
|Total_Infected|Total_population|InfectionPercentage|
+--------------+----------------+-------------------+
|      19169166|      6326421290|                0.3|
+--------------+----------------+-------------------+



In [115]:
#locally infected percentage
statewisetesting.show()
statewisetesting.printSchema()

+----------+--------------------+------------+--------+--------+
|      Date|               State|TotalSamples|Negative|Positive|
+----------+--------------------+------------+--------+--------+
|2020-04-17|Andaman and Nicob...|      1403.0|    1210|    12.0|
|2020-04-24|Andaman and Nicob...|      2679.0|    NULL|    27.0|
|2020-04-27|Andaman and Nicob...|      2848.0|    NULL|    33.0|
|2020-05-01|Andaman and Nicob...|      3754.0|    NULL|    33.0|
|2020-05-16|Andaman and Nicob...|      6677.0|    NULL|    33.0|
|2020-05-19|Andaman and Nicob...|      6965.0|    NULL|    33.0|
|2020-05-20|Andaman and Nicob...|      7082.0|    NULL|    33.0|
|2020-05-21|Andaman and Nicob...|      7167.0|    NULL|    33.0|
|2020-05-22|Andaman and Nicob...|      7263.0|    NULL|    33.0|
|2020-05-23|Andaman and Nicob...|      7327.0|    NULL|    33.0|
|2020-05-24|Andaman and Nicob...|      7327.0|    NULL|    33.0|
|2020-05-25|Andaman and Nicob...|      7363.0|    NULL|    33.0|
|2020-05-26|Andaman and N

In [116]:

# to find out the infected percentage in locally
covid_infected_locally_Infected_percentage = """
SELECT
    sum(TotalSamples) AS TotalSamples,
    sum(positive) AS PositiveCases,
    ROUND((SUM(positive) / SUM(TotalSamples) * 100), 2) AS Infection_percentage
FROM
    statewisetestings;
"""


spark.sql(covid_infected_locally_Infected_percentage).show()


+---------------+-------------+--------------------+
|   TotalSamples|PositiveCases|Infection_percentage|
+---------------+-------------+--------------------+
|8.7829949447E10| 3.20053246E8|                0.36|
+---------------+-------------+--------------------+




--3. To find out the countries with the highest infection rates

In [117]:
#showing the worldometer data and creating a tempview
worldometer.printSchema()
worldometer.show()


root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDe

In [118]:
#countries with highest infection rate
infected_highest = """
SELECT 
    `Country/Region`,
    SUM(TotalCases) AS TotalCases,
    SUM(Population) AS Population,
    ROUND((SUM(TotalCases) / SUM(Population)) * 100, 2) AS Infection_Rates
FROM 
    worldometer
GROUP BY 
    `Country/Region`
ORDER BY 
    Infection_Rates DESC
limit 10;
"""
spark.sql(infected_highest).show()


+--------------+----------+----------+---------------+
|Country/Region|TotalCases|Population|Infection_Rates|
+--------------+----------+----------+---------------+
|         Qatar|    112092|   2807805|           3.99|
| French Guiana|      8127|    299385|           2.71|
|       Bahrain|     42889|   1706669|           2.51|
|    San Marino|       699|     33938|           2.06|
|         Chile|    366671|  19132514|           1.92|
|        Panama|     71418|   4321282|           1.65|
|        Kuwait|     70045|   4276658|           1.64|
|          Oman|     80713|   5118446|           1.58|
|           USA|   5032179| 331198130|           1.52|
|  Vatican City|        12|       801|            1.5|
+--------------+----------+----------+---------------+



4. To find out the countries and continents with the highest death counts

In [119]:
#continenets with the highest death counts
worldometer.show()
worldometer.printSchema()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|    NULL|     162804|     NULL|       2576668|        NULL|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    NULL|      98644|     NULL|       2047660|        NULL|     771258|         

In [120]:
#continents with highest death counts
continent_highest_deaths_counts="""
select
	Continent,
	sum(TotalDeaths) as total_deaths
from
	worldometer
group by 
	Continent
order by 
	total_deaths desc;"""

spark.sql(continent_highest_deaths_counts).show()

+-----------------+------------+
|        Continent|total_deaths|
+-----------------+------------+
|    North America|      229855|
|           Europe|      205232|
|    South America|      154885|
|             Asia|      100627|
|           Africa|       22114|
|Australia/Oceania|         281|
|             NULL|          13|
+-----------------+------------+



In [121]:
#reading a file and showing a schema
covid_19_clean_complete.printSchema()
covid_19_clean_complete.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|      Province/State|     Country/Region|      Lat|      Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|                NULL|        Afghanistan| 33.93911| 67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|                NULL|            Albania|  41.1533|   20.1683|2020-01-22

In [122]:
#country with highest deaths
country_highest_deaths_count="""
select 
		`Country/Region`,
		sum(Deaths) as total_deaths
from 
	covid_19_clean_complete
group by
	`Country/Region`
order by
	total_deaths desc;"""
spark.sql(country_highest_deaths_count).show()

+--------------+------------+
|Country/Region|total_deaths|
+--------------+------------+
|            US|    11011411|
|United Kingdom|     3997775|
|        Brazil|     3938034|
|         Italy|     3707717|
|        France|     3048524|
|         Spain|     3033030|
|        Mexico|     1728277|
|         India|     1111831|
|          Iran|     1024136|
|       Belgium|      963679|
|       Germany|      871322|
|        Canada|      699566|
|         China|      672413|
|          Peru|      652113|
|   Netherlands|      622314|
|        Russia|      619385|
|        Turkey|      466056|
|        Sweden|      448913|
|       Ecuador|      346618|
|         Chile|      322480|
+--------------+------------+
only showing top 20 rows



5. Average number of deaths by day (Continents and Countries) 

In [123]:
#countries with highest death counts
covid_19_clean_complete.printSchema()
covid_19_clean_complete.show()


root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|      Province/State|     Country/Region|      Lat|      Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|                NULL|        Afghanistan| 33.93911| 67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|                NULL|            Albania|  41.1533|   20.1683|2020-01-22

In [124]:
#country average death perday
country_average_perDay_deaths="""
select 
	`Country/Region`,
	round(sum(deaths)/count(distinct date),2) as deaths_per_day_average
from 
	covid_19_clean_complete
group by 
	`Country/Region`
order by
	deaths_per_day_average desc;"""
spark.sql(country_average_perDay_deaths).show()

+--------------+----------------------+
|Country/Region|deaths_per_day_average|
+--------------+----------------------+
|            US|              58571.34|
|United Kingdom|              21264.76|
|        Brazil|              20946.99|
|         Italy|               19721.9|
|        France|              16215.55|
|         Spain|              16133.14|
|        Mexico|               9192.96|
|         India|               5913.99|
|          Iran|               5447.53|
|       Belgium|               5125.95|
|       Germany|               4634.69|
|        Canada|                3721.1|
|         China|               3576.66|
|          Peru|               3468.69|
|   Netherlands|               3310.18|
|        Russia|                3294.6|
|        Turkey|               2479.02|
|        Sweden|               2387.84|
|       Ecuador|               1843.71|
|         Chile|               1715.32|
+--------------+----------------------+
only showing top 20 rows



In [125]:
#reading a file and showing the schema
covid_19_clean_complete.printSchema()
covid_19_clean_complete.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|      Province/State|     Country/Region|      Lat|      Long|      Date|Confirmed|Deaths|Recovered|Active|          WHO Region|
+--------------------+-------------------+---------+----------+----------+---------+------+---------+------+--------------------+
|                NULL|        Afghanistan| 33.93911| 67.709953|2020-01-22|        0|     0|        0|     0|Eastern Mediterra...|
|                NULL|            Albania|  41.1533|   20.1683|2020-01-22

In [126]:
#deaths average per day for the continent
continent_average_perDay_deaths="""
select
	w.Continent,
	round(sum(c.Deaths)/count(distinct c.Date),2) as deaths_per_day_average
from 
	covid_19_clean_complete c
join 
	worldometer w 
on
	w.`Country/Region`=c.`Country/Region`
group by
	w.Continent
order by
	w.Continent;"""
spark.sql(continent_average_perDay_deaths).show()

+-----------------+----------------------+
|        Continent|deaths_per_day_average|
+-----------------+----------------------+
|           Africa|               3628.02|
|             Asia|              20654.07|
|Australia/Oceania|                 72.18|
|           Europe|              78134.58|
|    North America|              14181.08|
|    South America|              30222.54|
+-----------------+----------------------+



6. Average of cases divided by the number of population of each country (TOP 10) 

In [127]:
#reading a file and  showing schema
worldometer.printSchema()
worldometer.show()

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDe

In [128]:
#average cases in countries
average_cases_country="""
SELECT 
    `Country/Region`,
    ROUND( AVG(TotalCases) /  MAX(Population), 2) AS cases_per_population
FROM 
    Global_data
GROUP BY 
    `Country/Region`
ORDER BY 
    cases_per_population desc
limit 10;"""
spark.sql(average_cases_country).show()

+--------------+--------------------+
|Country/Region|cases_per_population|
+--------------+--------------------+
|         Qatar|                0.04|
| French Guiana|                0.03|
|       Bahrain|                0.03|
|    San Marino|                0.02|
|        Kuwait|                0.02|
|         Chile|                0.02|
|        Panama|                0.02|
|           USA|                0.02|
|          Oman|                0.02|
|     Singapore|                0.01|
+--------------+--------------------+

